In [1]:
import sys
import os
sys.path.append(os.path.dirname(os.getcwd()))
sys.path.append(os.path.dirname(os.path.dirname(os.getcwd())))
from lib.log import logger
from utils import load_pickle, save_pickle, summarize_distribution, find_rt_bound
import numpy as np

2022-10-21 10:55:17,766 Note: NumExpr detected 56 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2022-10-21 10:55:17,767 NumExpr defaulting to 8 threads.


In [2]:
# 为每个Sample子网中的每个用户随机生成其推文子网
# NOTE: 对于不足N=20个推文邻居的用户节点, 
# 1) 取其所有的推文节点加入子网, 剩余不足的在最后用其他用户的剩余推文来补全;
# 2) 如果此时仍然不够补全, 那么就舍弃这一用户子网(暂定)

Ntimestages = 8

In [13]:
for tidx in range(8):
    hs_old = load_pickle(f"../heter_samples_ratio1/{tidx}/heter_samples.p")
    logger.info(f"tidx={tidx}, len={len(hs_old)}")

2022-10-12 13:29:35,950 tidx=0, len=1458
2022-10-12 13:29:38,084 tidx=1, len=3025
2022-10-12 13:29:42,386 tidx=2, len=5908
2022-10-12 13:29:48,368 tidx=3, len=7939
2022-10-12 13:29:53,162 tidx=4, len=6240
2022-10-12 13:30:00,675 tidx=5, len=9509
2022-10-12 13:30:11,960 tidx=6, len=14557
2022-10-12 13:30:27,868 tidx=7, len=20605


In [ ]:
# for idx in range(Ntimestages):
#     hs = load_pickle(f"../heter_samples_ratio3/{idx}/heter_samples.p")
#     logger.info(f"len={len(hs)}, bincount={np.bincount(hs.labels)}, class_weight={len(hs)/2/np.bincount(hs.labels)}")

"""
NOTE: 1:1
2022-09-27 14:53:12,473 len=1458, bincount=[616 842], class_weight=[1.18344156 0.86579572]
2022-09-27 14:53:13,098 len=3025, bincount=[1344 1681], class_weight=[1.12537202 0.89976205]
2022-09-27 14:53:14,215 len=5908, bincount=[2686 3222], class_weight=[1.09977662 0.91682185]
2022-09-27 14:53:15,656 len=7939, bincount=[3675 4264], class_weight=[1.08013605 0.9309334 ]
2022-09-27 14:53:16,875 len=6240, bincount=[3193 3047], class_weight=[0.97713749 1.02395799]
2022-09-27 14:53:18,682 len=9509, bincount=[5385 4124], class_weight=[0.88291551 1.15288555]
2022-09-27 14:53:21,634 len=14557, bincount=[7816 6741], class_weight=[0.93123081 1.07973594]
2022-09-27 14:53:28,913 len=20605, bincount=[10552 10053], class_weight=[0.97635519 1.02481846]
"""

In [ ]:
# text_ids Group by stage
# diffusion_dict = load_pickle("/root/Lab_Related/data/Heter-GAT/Classic/ActionLog.p")

# upper_bs = {}
# for hashtag, cascades in diffusion_dict.items():
#     time_span = (cascades[-1][1]-cascades[0][1])/Ntimestages
#     upper_bs[hashtag] = [i*time_span+cascades[0][1] for i in range(1,Ntimestages+1)]

# graphid2tag = load_pickle("/root/Lab_Related/data/Heter-GAT/Classic/text/graphid2tag.p")
# graphid2timestamp = load_pickle("/root/Lab_Related/data/Heter-GAT/Classic/text/graphid2timestamp.p")

# text_ids = [set() for _ in range(Ntimestages)]
# for text_id, tag in graphid2tag.items():
#     timestamp = graphid2timestamp[text_id]
#     idx = find_rt_bound(timestamp, upper_bs[tag])
#     text_ids[idx].add(text_id)
#     # print(f"{timestamp:e}, {idx}, {upper_bs[tag]}")
# save_pickle(text_ids, "/root/Lab_Related/data/Heter-GAT/Classic/text/textids_groupbystage.p")

In [ ]:
# NOTE: User can only contact those tweets within appropriate stages

# user_texts = load_pickle("/root/Lab_Related/data/Heter-GAT/Classic/text/User-Text.p")
# textids_groupbystage = load_pickle("/root/Lab_Related/data/Heter-GAT/Classic/text/textids_groupbystage.p")
# text_ids = set()
# ut_mp = [{} for _ in range(Ntimestages)]
# for tidx in range(Ntimestages):
#     text_ids |= textids_groupbystage[tidx] # accumulate seen text_ids by stage
#     for user, texts in user_texts.items():
#         ut_mp[tidx][user] = sorted(list(text_ids & set(list(texts.keys()))))
# save_pickle(ut_mp, "/root/Lab_Related/data/Heter-GAT/Classic/text/utmp_groupbystage.p")

In [ ]:
# # Build T->U mp
# user_texts = load_pickle("/root/Lab_Related/data/Heter-GAT/Classic/text/User-Text.p")

# tweetid2userid_mp = {}
# for user, texts in user_texts.items():
#     for text_id in texts.keys():
#         tweetid2userid_mp[text_id] = user
# save_pickle(tweetid2userid_mp, "/root/Lab_Related/data/Heter-GAT/Classic/text/tweetid2userid_mp.p")

In [3]:
# Random Choose N=20 Tweets Per User
import numpy as np
from utils import SubGraphSample

# from_dirname = "/root/data/HeterGAT/stages/stages_subg483_inf_40_1718027_deg_18_483_ego_50_neg_1_restart_20"
from_dirname = "/root/data/HeterGAT/stages/stages_subg483_inf_40_1718027_deg_18_483_ego_20_neg_1_restart_20"
# to_dirname   = "/root/data/HeterGAT/stages/hs_subg483_inf_40_1718027_deg_18_483_ego_50_neg_1_restart_20"
to_dirname   = "/root/data/HeterGAT/stages/hs_subg483_inf_40_1718027_deg_18_483_ego_20_neg_1_restart_20"
tweets_per_user = 5
ut_mp = load_pickle("/root/data/HeterGAT/basic/text/utmp_groupbystage.p")
tweetid2userid_mp = load_pickle("/root/data/HeterGAT/basic/text/tweetid2userid_mp.p")

def read_subgraphsamples(samples_dirpath):
    samples = SubGraphSample()
    samples.adj_matrices = np.load(os.path.join(samples_dirpath, "adjacency_matrix.npy"))
    samples.influence_features = np.load(os.path.join(samples_dirpath, "influence_feature.npy"))
    samples.vertex_ids = np.load(os.path.join(samples_dirpath, "vertex_id.npy"))
    samples.labels = np.load(os.path.join(samples_dirpath, "label.npy"))
    samples.tags = np.load(os.path.join(samples_dirpath, "hashtag.npy"))
    samples.time_stages = np.load(os.path.join(samples_dirpath, "stage.npy"))
    return samples

def gen_random_tweets(samples: SubGraphSample):
    tweet_ids = []
    sample_ids = []

    for idx in range(len(samples.labels)):
        if idx and idx % 10000 == 0:
            logger.info(f"idx={idx}, sample_ids={len(sample_ids)}, tweet_ids={len(tweet_ids)}")
        stage = samples.time_stages[idx]
        selected_tweet_ids  = set()
        candidate_tweet_ids = set()
        for vertex_id in samples.vertex_ids[idx]:
            available_tweet_ids = ut_mp[stage][vertex_id]
            random_ids = np.random.choice(available_tweet_ids, size=min(tweets_per_user, len(available_tweet_ids)), replace=False)
            selected_tweet_ids  |= set(random_ids)
            candidate_tweet_ids |= set(available_tweet_ids)-set(random_ids)
        candidate_tweet_ids -= selected_tweet_ids
        # logger.info(f"Length: sample={len(selected_tweet_ids)}, remain={len(candidate_tweet_ids)}, expected={len(samples.vertex_ids[idx])*tweets_per_user}")

        if len(selected_tweet_ids) != len(samples.vertex_ids[idx])*tweets_per_user:
            diff = len(samples.vertex_ids[idx])*tweets_per_user - len(selected_tweet_ids)
            if diff > len(candidate_tweet_ids):
                continue
            selected_tweet_ids |= set(np.random.choice(list(candidate_tweet_ids), size=diff, replace=False))
        sample_ids.append(idx)
        tweet_ids.append(selected_tweet_ids)
    logger.info(f"Finish Sampling Random Tweets... sample_ids={len(sample_ids)}, tweet_ids={len(tweet_ids)}")

    os.makedirs(to_dirname, exist_ok=True)
    selected_samples = SubGraphSample(
        adj_matrices=samples.adj_matrices[sample_ids],
        influence_features=samples.influence_features[sample_ids],
        vertex_ids=samples.vertex_ids[sample_ids],
        labels=samples.labels[sample_ids],
        tags=samples.tags[sample_ids],
        time_stages=samples.time_stages[sample_ids],
    )
    save_pickle(sample_ids, f"{to_dirname}/sample_ids.p")
    save_pickle(tweet_ids, f"{to_dirname}/tweet_ids.p")
    save_pickle(selected_samples, f"{to_dirname}/selected_samples.p")
    logger.info("Finish Saving pkl...")

# for tidx in range(Ntimestages):
#     samples = read_subgraphsamples(f"{sp_dirname}/{tidx}/")
#     gen_random_tweets(samples)

samples = read_subgraphsamples(samples_dirpath=from_dirname)
gen_random_tweets(samples=samples)

"""
2022-10-21 10:56:51,140 idx=10000, sample_ids=9430, tweet_ids=9430
2022-10-21 10:57:04,385 idx=20000, sample_ids=18175, tweet_ids=18175
2022-10-21 10:57:18,214 idx=30000, sample_ids=28041, tweet_ids=28041
2022-10-21 10:57:30,377 idx=40000, sample_ids=37383, tweet_ids=37383
2022-10-21 10:57:43,875 idx=50000, sample_ids=46835, tweet_ids=46835
2022-10-21 10:57:57,680 idx=60000, sample_ids=56571, tweet_ids=56571
2022-10-21 10:57:59,849 Finish Sampling Random Tweets... sample_ids=58185, tweet_ids=58185
2022-10-21 10:58:18,422 Finish Saving pkl...
"""

2022-10-21 10:56:51,140 idx=10000, sample_ids=9430, tweet_ids=9430
2022-10-21 10:57:04,385 idx=20000, sample_ids=18175, tweet_ids=18175
2022-10-21 10:57:18,214 idx=30000, sample_ids=28041, tweet_ids=28041
2022-10-21 10:57:30,377 idx=40000, sample_ids=37383, tweet_ids=37383
2022-10-21 10:57:43,875 idx=50000, sample_ids=46835, tweet_ids=46835
2022-10-21 10:57:57,680 idx=60000, sample_ids=56571, tweet_ids=56571
2022-10-21 10:57:59,849 Finish Sampling Random Tweets... sample_ids=58185, tweet_ids=58185
2022-10-21 10:58:18,422 Finish Saving pkl...


In [ ]:
"""
NOTE: 1:1
2022-09-26 20:37:04,459 Length: users=9482, tweets=1458
2022-09-26 20:37:27,938 Length: users=5201, tweets=3025
2022-09-26 20:38:04,250 Length: users=7358, tweets=5908
2022-09-26 20:39:01,396 Length: users=8091, tweets=7939
2022-09-26 20:40:06,630 Length: users=6342, tweets=6240
2022-09-26 20:41:22,722 Length: users=9583, tweets=9509
2022-09-26 20:43:33,540 Length: users=14592, tweets=14557
2022-09-26 20:47:39,127 Length: users=20605, tweets=20605
"""

"""
NOTE: 1:3
2022-09-27 10:50:53,798 Length: users=18071, tweets=2705
2022-09-27 10:51:22,941 Length: users=9698, tweets=5633
2022-09-27 10:52:14,377 Length: users=13823, tweets=11092
2022-09-27 10:53:32,810 Length: users=15253, tweets=14905
2022-09-27 10:54:58,749 Length: users=12441, tweets=12184
2022-09-27 10:56:45,734 Length: users=19229, tweets=19051
2022-09-27 10:59:52,396 Length: users=29467, tweets=29380
2022-09-27 11:05:41,109 Length: users=41437, tweets=41436
"""

"""
NOTE: stages_lab_try2_inf_199_898012_deg_18_9265_ego_50_neg_1_restart_20, ratio=1:1
2022-10-10 09:40:33,418 idx=10000, tweet_ids len=9209
2022-10-10 09:41:09,871 idx=20000, tweet_ids len=18038
2022-10-10 09:42:02,105 idx=30000, tweet_ids len=27475
2022-10-10 09:42:42,801 idx=40000, tweet_ids len=36228
2022-10-10 09:43:25,821 idx=50000, tweet_ids len=44930
2022-10-10 09:43:57,018 idx=60000, tweet_ids len=52328
2022-10-10 09:44:37,991 idx=70000, tweet_ids len=62225
2022-10-10 09:45:05,098 idx=80000, tweet_ids len=69763
2022-10-10 09:45:50,042 idx=90000, tweet_ids len=79647
2022-10-10 09:46:18,392 idx=100000, tweet_ids len=87068
2022-10-10 09:47:12,400 idx=110000, tweet_ids len=96583
2022-10-10 09:48:01,686 idx=120000, tweet_ids len=106062
2022-10-10 09:48:36,812 idx=130000, tweet_ids len=113834
2022-10-10 09:49:28,524 idx=140000, tweet_ids len=123752
2022-10-10 09:50:05,030 idx=150000, tweet_ids len=132095
2022-10-10 09:50:46,975 idx=160000, tweet_ids len=141435
2022-10-10 09:51:23,820 idx=170000, tweet_ids len=150030
2022-10-10 09:52:18,778 idx=180000, tweet_ids len=159629
2022-10-10 09:52:44,353 idx=190000, tweet_ids len=167512
2022-10-10 09:53:24,533 idx=200000, tweet_ids len=175910
2022-10-10 09:54:02,472 idx=210000, tweet_ids len=185350
2022-10-10 09:55:04,512 idx=220000, tweet_ids len=195349
2022-10-10 09:55:44,417 idx=230000, tweet_ids len=203606
2022-10-10 09:56:19,747 idx=240000, tweet_ids len=210787
2022-10-10 09:57:00,054 idx=250000, tweet_ids len=219523
2022-10-10 09:57:36,717 idx=260000, tweet_ids len=227820
2022-10-10 09:58:16,037 idx=270000, tweet_ids len=236144
2022-10-10 09:58:52,221 idx=280000, tweet_ids len=242976
2022-10-10 09:59:22,469 idx=290000, tweet_ids len=251292
2022-10-10 10:00:02,073 Length: users=297006, tweets=258046
"""

"""
NOTE: stages_lab_try2_inf_199_898012_deg_18_9265_ego_50_neg_1_restart_20, ratio=1:1, only stage-7
2022-10-10 13:46:45,260 idx=40000, tweet_ids len=12203
2022-10-10 13:47:21,965 idx=70000, tweet_ids len=18090
2022-10-10 13:47:42,965 idx=90000, tweet_ids len=21608
2022-10-10 13:48:08,042 idx=110000, tweet_ids len=25582
2022-10-10 13:48:20,330 idx=120000, tweet_ids len=27482
2022-10-10 13:49:10,263 idx=160000, tweet_ids len=35467
2022-10-10 13:50:20,411 idx=220000, tweet_ids len=46816
2022-10-10 13:51:47,310 Length: users=297006, tweets=60957
"""

In [4]:
# Adj -> Heter-Adj
from typing import List
import numpy as np
from utils import SubGraphSample, HeterSubGraphSample
import copy
from scipy import sparse

# from_dirname = "/root/data/HeterGAT/stages/stages_subg483_inf_40_1718027_deg_18_483_ego_50_neg_1_restart_20"
from_dirname = "/root/data/HeterGAT/stages/stages_subg483_inf_40_1718027_deg_18_483_ego_20_neg_1_restart_20"
# to_dirname   = "/root/data/HeterGAT/stages/hs_subg483_inf_40_1718027_deg_18_483_ego_50_neg_1_restart_20"
to_dirname   = "/root/data/HeterGAT/stages/hs_subg483_inf_40_1718027_deg_18_483_ego_20_neg_1_restart_20"
tweetid2userid_mp = load_pickle("/root/data/HeterGAT/basic/text/tweetid2userid_mp.p")
tweet_feat = np.array(load_pickle("/root/data/HeterGAT/basic/doc2topic_tweetfeat.p"))
tot_users_nb = 208894
logger.info("Finish Loading Data...")

def extend_adj_matrices(samples: SubGraphSample, tweet_ids: List[List[int]], heter_samples: HeterSubGraphSample):
    adj_matrices = samples.adj_matrices
    vertex_ids   = samples.vertex_ids

    for idx in range(len(samples.labels)):
        if idx and idx % 1000 == 0:
            logger.info(f"idx={idx}")
        extend_subnetwork = np.array(
            np.concatenate((vertex_ids[idx], np.array(list(tweet_ids[idx])))), 
            dtype=int
        )
        heter_samples.vertex_ids.append(extend_subnetwork) # (N,)
        subnetwork_size, user_ids_nb = len(extend_subnetwork), len(vertex_ids[idx])
        elem_idx_mp = {elem:idx for idx,elem in enumerate(extend_subnetwork)}

        # Build U-U, U-T adj-matrices, respectively
        heter_adj_matrices = []
        uu_matrices = np.array([[0]*subnetwork_size for _ in range(subnetwork_size)], dtype=int)
        uu_matrices[:user_ids_nb,:user_ids_nb] = adj_matrices[idx]
        heter_adj_matrices.append(sparse.csr_matrix(uu_matrices)) # toarray()

        # Get Corresponding User_id By Tweet_id, and then convert them into indexes in extend_subnetwork
        ut_matrices = copy.deepcopy(uu_matrices)
        for tweet_id in tweet_ids[idx]:
            user_id = tweetid2userid_mp[tweet_id]
            net_userid = elem_idx_mp[user_id]
            net_tweetid = elem_idx_mp[tweet_id]
            ut_matrices[net_userid][net_tweetid] = 1
        heter_adj_matrices.append(sparse.csr_matrix(ut_matrices)) # toarray()
        del uu_matrices, ut_matrices

        heter_samples.heter_adj_matrices.append(heter_adj_matrices) # (|Rs|,N,N)
    
    heter_samples.labels = samples.labels

def extend_initial_features(samples: SubGraphSample, heter_samples: HeterSubGraphSample):
    influence_features = samples.influence_features
    stage = samples.time_stages[0]
    hashtags = samples.tags

    for idx in range(len(samples)):
        vertex_ids = heter_samples.vertex_ids[idx]
        # Build User Features
        # initial_features(user) = influence_features||Gra(hashtag,stage)||Exp(hashtag,stage)||Cas(hashtag,stage), dim=2
        gra_feat = np.array(load_pickle(f"/root/data/HeterGAT/user_features/norm_gravity_feature/hashtag{hashtags[idx]}_t{stage}.p")).reshape(-1,1)
        exp_feat = np.array(load_pickle(f"/root/data/HeterGAT/user_features/norm_exptime_feature1/hashtag{hashtags[idx]}_t{stage}.p")).reshape(-1,1)
        cas_feat = np.array(load_pickle(f"/root/data/HeterGAT/user_features/norm_ce_feature/hashtag{hashtags[idx]}_t{stage}.p")).reshape(-1,1)
        user_ids = samples.vertex_ids[idx]
        user_features = np.concatenate((influence_features[idx], gra_feat[user_ids], exp_feat[user_ids], cas_feat[user_ids]), axis=1)

        # Build Tweet Features
        tweet_ids = vertex_ids[len(user_ids):]
        tweet_features = tweet_feat[tweet_ids]

        # Extend Two Features with Padding zeros
        initial_features = np.concatenate((
            np.append(user_features,  [[0]*tweet_features.shape[1] for _ in range(user_features.shape[0])],  1),
            np.append([[0]*user_features.shape[1]  for _ in range(tweet_features.shape[0])], tweet_features, 1)
        ), axis=0)
        heter_samples.initial_features.append(initial_features)

# Main Solution
tweetids = load_pickle(f"{to_dirname}/tweet_ids.p")
samples  = load_pickle(f"{to_dirname}/selected_samples.p")
hetersamples = HeterSubGraphSample()
extend_adj_matrices(samples, tweetids, hetersamples)
extend_initial_features(samples, hetersamples)

# Extend Vertex Indices
for idx in range(len(hetersamples)):
    vertex_ids = hetersamples.vertex_ids[idx]
    hetersamples.vertex_ids[idx] = [index if index<tot_users_nb else index+tot_users_nb for index in vertex_ids]

# 
hetersamples.heter_adj_matrices = np.array(hetersamples.heter_adj_matrices)
hetersamples.initial_features = np.array(hetersamples.initial_features)
hetersamples.vertex_ids = np.array(hetersamples.vertex_ids)
hetersamples.labels = np.array(hetersamples.labels)

save_pickle(hetersamples, f"{to_dirname}/heter_samples.p")

# for tidx in range(Ntimestages):
#     tweetids = load_pickle(f"{to_dirname}/{tidx}/tweet_ids.p")
#     samples  = load_pickle(f"{to_dirname}/{tidx}/samples.p")
#     hetersamples = HeterSubGraphSample()
#     extend_adj_matrices(samples, tweetids, hetersamples)
#     extend_initial_features(samples, hetersamples)

#     # Extend Vertex Indices
#     for idx in range(len(hetersamples)):
#         vertex_ids = hetersamples.vertex_ids[idx]
#         hetersamples.vertex_ids[idx] = [index if index<tot_users_nb else index+tot_users_nb for index in vertex_ids]
    
#     # 
#     hetersamples.heter_adj_matrices = np.array(hetersamples.heter_adj_matrices)
#     hetersamples.initial_features = np.array(hetersamples.initial_features)
#     hetersamples.vertex_ids = np.array(hetersamples.vertex_ids)
#     hetersamples.labels = np.array(hetersamples.labels)

#     # save_pickle(hetersamples, f"../{hetersp_dirname}/{tidx}/heter_samples.p")
#     break

2022-10-21 11:04:19,113 Finish Loading Data...
2022-10-21 11:04:24,026 idx=1000
2022-10-21 11:04:25,830 idx=2000
2022-10-21 11:04:27,537 idx=3000
2022-10-21 11:04:29,272 idx=4000
2022-10-21 11:04:30,942 idx=5000
2022-10-21 11:04:32,671 idx=6000
2022-10-21 11:04:34,358 idx=7000
2022-10-21 11:04:36,023 idx=8000
2022-10-21 11:04:37,700 idx=9000
2022-10-21 11:04:39,386 idx=10000
2022-10-21 11:04:41,025 idx=11000
2022-10-21 11:04:42,695 idx=12000
2022-10-21 11:04:44,403 idx=13000
2022-10-21 11:04:46,104 idx=14000
2022-10-21 11:04:47,803 idx=15000
2022-10-21 11:04:49,499 idx=16000
2022-10-21 11:04:51,180 idx=17000
2022-10-21 11:04:52,868 idx=18000
2022-10-21 11:04:54,527 idx=19000
2022-10-21 11:04:56,185 idx=20000
2022-10-21 11:04:57,879 idx=21000
2022-10-21 11:04:59,537 idx=22000
2022-10-21 11:05:01,207 idx=23000
2022-10-21 11:05:02,877 idx=24000
2022-10-21 11:05:04,523 idx=25000
2022-10-21 11:05:06,210 idx=26000
2022-10-21 11:05:16,612 idx=27000
2022-10-21 11:05:18,293 idx=28000
2022-10-21

KeyboardInterrupt: 

In [32]:
# # TODO: Add initial_features in HeterSubGraphSample

# import numpy as np
# from utils import SubGraphSample, HeterSubGraphSample

# tweet_feat = np.array(load_pickle("heter-training/doc2topic_tweetfeat.p"))

# def extend_initial_features(samples: SubGraphSample, heter_samples: HeterSubGraphSample):
#     influence_features = samples.influence_features
#     stage = samples.time_stages[0]
#     hashtags = samples.tags

#     if len(heter_samples.initial_features):
#         heter_samples.initial_features = []

#     for idx in range(len(samples)):
#         vertex_ids = heter_samples.vertex_ids[idx]
#         # Build User Features
#         # initial_features(user) = influence_features||Gra(hashtag,stage)||Exp(hashtag,stage)||Cas(hashtag,stage), dim=2
#         gra_feat = np.array(load_pickle(f"user_features/norm_gravity_feature/hashtag{hashtags[idx]}_t{stage}.p")).reshape(-1,1)
#         exp_feat = np.array(load_pickle(f"user_features/norm_exptime_feature1/hashtag{hashtags[idx]}_t{stage}.p")).reshape(-1,1)
#         cas_feat = np.array(load_pickle(f"user_features/norm_ce_feature/hashtag{hashtags[idx]}_t{stage}.p")).reshape(-1,1)
#         user_ids = samples.vertex_ids[idx]
#         user_features = np.concatenate((influence_features[idx], gra_feat[user_ids], exp_feat[user_ids], cas_feat[user_ids]), axis=1)

#         # Build Tweet Features
#         tweet_ids = vertex_ids[len(user_ids):]
#         tweet_features = tweet_feat[tweet_ids]

#         # Extend Two Features with Padding zeros
#         initial_features = np.concatenate((
#             np.append(user_features,  [[0]*tweet_features.shape[1] for _ in range(user_features.shape[0])],  1),
#             np.append([[0]*user_features.shape[1]  for _ in range(tweet_features.shape[0])], tweet_features, 1)
#         ), axis=0)
#         heter_samples.initial_features.append(initial_features)
    
# for tidx in range(Ntimestages):
#     samples = load_pickle(f"heter_samples/{tidx}/samples.p")
#     heter_samples = load_pickle(f"heter_samples/{tidx}/heter_samples_extendhadj.p")
#     extend_initial_features(samples, heter_samples)
#     save_pickle(heter_samples, f"heter_samples/{tidx}/heter_samples_extendfeat.p")


In [3]:
# # Extend Hadj to Matrices
# # 208894
# import numpy as np

# Ntimestages = 8
# for tidx in range(Ntimestages):
#     heter_samples = load_pickle(f"heter_samples/{tidx}/heter_samples_extendfeat.p")

#     for idx in range(len(heter_samples)):
#         vertex_ids = heter_samples.vertex_ids[idx]
#         extended_vertex_ids = np.concatenate((vertex_ids[:50], np.array([elem+208894 for elem in vertex_ids[50:]])), axis=0)
#         heter_samples.vertex_ids[idx] = extended_vertex_ids

#     save_pickle(heter_samples, f"heter_samples/{tidx}/heter_samples_extendidx.p")

# # NOTE: Time spent: 20min42.7s

In [ ]:
# for idx in range(Ntimestages):
#     heter_samples = load_pickle(f"heter_samples/{idx}/heter_samples_extendidx.p")
#     heter_samples.heter_adj_matrices = np.array(heter_samples.heter_adj_matrices)
#     heter_samples.initial_features = np.array(heter_samples.initial_features)
#     heter_samples.vertex_ids = np.array(heter_samples.vertex_ids)
#     save_pickle(heter_samples, f"heter_samples/{idx}/heter_samples.p")

# NOTE: Time spent: 26min28.9s

In [ ]:
# NOTE: Convert to Tensors

# import torch

# Ntimestages = 8
# for tidx in range(Ntimestages):
#     heter_samples = load_pickle(f"heter_samples/{tidx}/heter_samples.p")
#     heter_samples.vertex_ids = torch.Tensor(heter_samples.vertex_ids)
#     heter_samples.labels = torch.Tensor(heter_samples.labels)
#     heter_samples.initial_features = torch.Tensor(heter_samples.initial_features)
#     heter_samples.heter_adj_matrices = torch.stack([
#         torch.stack([get_sparse_tensor(adj.tocoo()) for adj in heter_samples.heter_adj_matrices[:,hadj_idx]])
#     for hadj_idx in range(heter_samples.heter_adj_matrices.shape[1])])

#     save_pickle(heter_samples, f"heter_samples/{tidx}/heter_samples_tensor.p")

In [ ]:
# [132641, 58655, 57581, 57830, 58903, 75070, 100318, 147189]